# ElectroMKM class import and object instantiation

In [1]:
import sys
sys.path.insert(0, "../") 
from electromkm import electroMKM

The demo system deal with the Hydrogen Evolution Reaction (HER) with random values.

In [2]:
model = electroMKM('CO2R_110',
                   'rm.mkm', 
                   'g.mkm', 
                    t_ref=298)

['C2C 1: C1O2(g) + 2H2O(e) + K+(cat) -> C1O1(g) + H2O(g)', 'HER 5: 2H2O(e) + K+(cat) -> H2(g)', '', '', '', 'C1O2(g) + i000000  -> i102101a', 'C1O1(g) + i000000 -> i101101', 'i112102 + H2O(e) -> i101101 + H2O(g)', 'i102101a + H2O(e)  -> i112102', 'H2O(e) + i000000  -> i010101', 'H2(g) + 2i000000 -> 2i010101']
K+
K+


In [3]:
model.NC_base

1

# Model exploration


To investigate the characteristics of the system under study, several attributes can be easily inspected to check general information like number of elementary reactions, energetics, reaction network, etc.

In [4]:
print(model.species_gas)

['C1O2(g)', 'C1O1(g)', 'H2O(g)', 'H2(g)']


### Defined species in the system

N.B. H(e) is used to define H+ + e-.

In [5]:
model.species_tot

['i000000',
 'i010101',
 'i101101',
 'i102101a',
 'i112102',
 'H2O(e)',
 'C1O2(g)',
 'C1O1(g)',
 'H2O(g)',
 'H2(g)']

### Visualize Gibbs energetics of the system

Reaction types: 'ads'=adsorption
                'des'=desorption
                'sur'=surface reaction. 
The suffix "+e" means that that elementary reaction is a charge-transfer step.

In [6]:
model.df_gibbs

,,DGR / eV,DG barrier / eV,DG reverse barrier / eV
R1,ads,-0.039,0.000,0.039
R2,ads,0.259,0.259,0.000
R3,des+b_e,0.195,0.753,0.558
R4,sur+b_e,0.394,1.207,0.813
R5,sur+b_e,0.292,1.293,1.001
R6,ads,0.584,0.584,0.000


In [7]:
model.dh_barrier

array([0.   , 0.   , 0.767, 1.216, 1.297, 0.254])

### Stoichiometric matrix of the reaction network

In [8]:
model.df_system

,R1,R2,R3,R4,R5,R6
,ads,ads,des+b_e,sur+b_e,sur+b_e,ads
species,,,,,,
i000000,-1,-1,0,0,-1,-2
i010101,0,0,0,0,1,2
i101101,0,1,1,0,0,0
i102101a,1,0,0,-1,0,0
i112102,0,0,-1,1,0,0
H2O(e),0,0,-1,-1,-1,0
C1O2(g),-1,0,0,0,0,0
C1O1(g),0,-1,0,0,0,0


### Stoichiometric vector of the global reactions

In [9]:
model.gr_string

['C1O2(g) + 2H2O(e) + K+(cat) -> C1O1(g) + H2O(g)',
 '2H2O(e) + K+(cat) -> H2(g)']

model.stoich_numbers tells us that the first elementary reaction must be multiplied by two and summed up to the second one in order to get the global reaction. This is useful for checking the thermodynamic consistency of the developed models.

# Microkinetic runs and Tafel plot

Up to now, it is possible to run steady state runs via the electroMKM.kinetic_run() function.
The main inputs that must be provided are the applied overpotential and the pH of the electrolyte solution.
The output of the function is a Python dictionary containing information related to the performed simulation.

In [10]:
model.set_ODE_params(t_final=100)

Final integration time = 100s
Relative tolerance = 1e-12
Absolute tolerance = 1e-64


'Changed ODE solver parameters.'

### Steady state simulation

In [11]:
help(electroMKM.kinetic_run)

Help on function kinetic_run in module electromkm:

kinetic_run(self, overpotential:float, pH:float, cation_conc:float, potential_dl:float, initial_sur_coverage:list=None, temperature:float=298.0, pressure:float=100000.0, gas_composition:numpy.ndarray=None, verbose:int=0, jac:bool=False)
    Simulates a steady-state electrocatalytic run at the defined operating conditions.        
    Args:
        overpotential(float): applied overpotential [V vs SHE].
        pH(float): pH of the electrolyte solution [-].
        cation_conc(float): cation_conc at OHP [M]
        potential_dl (float): potential difference across double layer [V]
        temperature(float): Temperature of the system [K].
        pressure(float): Absolute pressure of the system [Pa].
        initial_conditions(nparray): Initial surface coverage array[-].
        verbose(int): 0=print all output; 1=print nothing.        
    Returns:
        (dict): Report of the electrocatalytic simulation.



In [10]:
import numpy as np
array=[]
for i in model.species_gas:
    if i=='C1O2(g)':
        array.append(0.0609)
    elif i=='H2O(g)':
        array.append(1)
    else:
        array.append(0)
array=np.array(array)
print(array)
exp = model.kinetic_run(-1.8, 6.54, 5.38, (1.8-0.8-0.28)/2, gas_composition = array , jac = True)

[0.0609 0.     1.     0.    ]
CO2R_110: Microkinetic run
Overpotential = -1.8V vs SHE    pH = 6.54
Overpotential = -1.4141400000000002V vs RHE
Temperature = 298.0K    Pressure = 1.0bar
cation_ads_ener: 0.31, site_adj_factor: 0.000
cation_ads_ener: 0.31, site_adj_factor: 0.000
final adjust_factor is : [3.3253038484259455e-05, 3.3253038484259455e-05]

C2C Current density: 9.25e+01 mA cm-2
C2C Selectivity: 94.06%
Most Abundant Surface Intermediate: i000000 Coverage: 78.22% 
CPU time: 0.21 s


In [11]:
exp['j_HER']

58.40109148697867

In [13]:
import numpy as np
array=[]
for i in model.species_gas:
    if i=='C1O2(g)':
        array.append(0.014)
    elif i=='H2O(g)':
        array.append(1)
    else:
        array.append(0)
array=np.array(array)
print(array)
exp = model.kinetic_run(-2.1, 8.3, 7.8134,(1.28-0.28)/2, gas_composition = array , jac = True)

[0.014 0.    1.    0.   ]
CO2R_110: Microkinetic run
Overpotential = -2.1V vs SHE    pH = 8.3
Overpotential = -1.6103V vs RHE
Temperature = 298.0K    Pressure = 1.0bar
cation_ads_ener: 0.28, site_adj_factor: 0.000
cation_ads_ener: 0.28, site_adj_factor: 0.000
final adjust_factor is : [0.00014367487876996396, 0.00014367487876996396]

C2C Current density: 3.74e+04 mA cm-2
C2C Selectivity: 78.28%
Most Abundant Surface Intermediate: i000000 Coverage: 93.46% 
CPU time: 0.18 s


In [17]:
model.target_label

'C2C'

In [18]:
model.gpcet_dict[model.target_label]


2.0

In [28]:
with open('output', 'w') as outfile:
    outfile.write('Current Density (mA/cm2) of {} {:.4e}\n'.format(model.target_label,model.gpcet_dict[model.target_label]*exp[str('j_'+model.target_label)]/10))
    for label in model.by_products_label:
        outfile.write('Current Density (mA/cm2) of {} {:.4e}'.format(label,model.gpcet_dict[label]*exp[str('j_'+label)]/10))

In [ ]:
exp['ddt']

Once steady state conditions have been checked, the solution can be easily analyzed. the main output consists of steady state surface coverage and reaction rate in term of current density.

In [14]:
exp['theta']

{'i000000': 0.9345644494374523,
 'i010101': 0.00602511935390456,
 'i101101': 0.00013084012014704787,
 'i102101a': 0.05914875096835147,
 'i112102': 0.00013084012014704787}

In [ ]:
exp['r']

Negative current density means reduction is occurring, while positive values means that reaction is evolving in the opposite direction. Values of current density are stored in mA cm-2.

In [ ]:
exp['j_HER']

### Tafel plot

In [ ]:
import numpy as np
n = np.arange(-1,-2.5,-0.25)
#n = np.array([-1.5,-1.75,-2,-2.25])
print(n)

In [ ]:
help(electroMKM.tafel_plot)

In [ ]:
model.tafel_plot("C2C", n ,7,3.7, gas_composition = array, jac=True)

In [ ]:
rxn_list = ['C2C','HER']
label = ['CO','H2']#,'Methane']

color = ['r','g']

model.j_potential(rxn_list, label, n ,7, color, gas_composition = array, jac=True,set_y_lim=[-10,25])

In [ ]:
model.drc_full('C2C',-1.6, 7,1, gas_composition = array,jac='True')

### Check for the analytical Jacobian matrix

In order to check if the implemented analytical Jacobian is correct or not, we run the same simulation with and without the analytical Jacobian: If the solutions are the same, the Jacobian is correct. If not, it means that the wrong Jacobian drives the system away from the correct solution. 

#### Simulation without analytical Jacobian

In [ ]:
exp1 = model.kinetic_run(-0.1, 7, jac=False)

#### Simulation with analytical Jacobian

In [ ]:
exp2 = model.kinetic_run(-0.1, 7, jac=True)

Observe the difference in the CPU time required to integrate the system without and with the Jacobian!